
# Неделя 7. Natural Language Processing 
## Классификация текста с помощью BERT

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# !rm -f /root/.kaggle/*
# !mkdir /root/.kaggle
# !cp /content/drive/MyDrive/kaggle.json /root/.kaggle

In [ ]:
# иногда требуется для работы токенизатора
# !pip install sentencepiece 

In [ ]:
!unzip -qq /content/news.csv.zip

In [ ]:
# устанавливаем библиотеку со множеством моделей huggingface.co/
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 4.2 MB/s 
     |████████████████████████████████| 6.6 MB 93.5 MB/s 
     |████████████████████████████████| 596 kB 77.1 MB/s 
     |████████████████████████████████| 86 kB 6.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# # скачиваем датасет
# !kaggle datasets download -d datatattle/covid-19-nlp-text-classification

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch

# импортируем трансформеры
import transformers
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import re

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


## Загружаем датасет с настоящими новостями, выбираем новости из Lenta

In [ ]:
news = pd.read_csv('/content/news.csv')
news

,source,title,text,publication_date,rubric,subrubric,tags
0,lenta.ru,Синий богатырь,В 1930-е годы Советский Союз охватила лихорадк...,2020-08-30T00:01:00+03:00,Экономика,Госэкономика,NaN
1,lenta.ru,Загитова согласилась вести «Ледниковый период»,Олимпийская чемпионка по фигурному катанию Ал...,2020-08-31T20:04:00+03:00,Спорт,Зимние виды,NaN
2,lenta.ru,Объяснена опасность однообразного питания,Российский врач-диетолог Римма Мойсенко объясн...,2020-08-31T20:07:00+03:00,Из жизни,Еда,NaN
3,lenta.ru,«Предохраняться? А зачем?»,В 2019 году телеканал «Ю» запустил адаптацию з...,2020-08-30T00:04:00+03:00,Интернет и СМИ,ТВ и радио,NaN
4,lenta.ru,Ефремов систематически употреблял наркотики,Актер Михаил Ефремов систематически употребл...,2020-08-31T18:27:00+03:00,Культура,Кино,NaN
...,...,...,...,...,...,...,...
21668,tjournal.ru,\n Россия прекратил...,\n\n \n \n \n \n \n...,1578056674,NaN,NaN,NaN
21669,tjournal.ru,\n Во Владивостоке ...,\n\n \n \n \n \n \n...,1577866951,NaN,NaN,NaN
21670,tjournal.ru,\n Дым от австралий...,\n\n \n \n \n \n \n...,1577864124,NaN,NaN,NaN
21671,tjournal.ru,\n Около 200 жителе...,\n\n \n \n \n \n \n...,1577894168,NaN,NaN,NaN


In [ ]:
news = news[news['source'] == 'lenta.ru']

In [ ]:
def clean(text):
    # text = text.lower()
    text = re.sub('\n{1,}', '', text)
    text = re.sub('\s{2,}', ' ', text)

    return text

In [ ]:
news['text'] = news['text'].apply(clean)

In [ ]:
news['fake'] = 0

In [ ]:
news[['text', 'fake']]

,text,fake
0,В 1930-е годы Советский Союз охватила лихорадк...,0
1,Олимпийская чемпионка по фигурному катанию Али...,0
2,Российский врач-диетолог Римма Мойсенко объясн...,0
3,В 2019 году телеканал «Ю» запустил адаптацию з...,0
4,Актер Михаил Ефремов систематически употреблял...,0
...,...,...
4505,C 1 января Болгария начала получать российский...,0
4506,В России вступили в силу новые правила содержа...,0
4507,Российский певец Сергей Лазарев объяснил свой ...,0
4508,Президент США Дональд Трамп заявил о нежелании...,0


## Загружаем новости, которые были сгенерены на предыдущем этапе с помощью GPT-3

In [ ]:
fake_news = pd.read_csv('/content/fake.csv')

In [ ]:
# fake_news = fake_news[fake_news['source'] == 'meduza.io']
# fake_news['text'] = fake_news['text'].apply(clean)
fake_news['fake'] = 1

In [ ]:
fake_news.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
fake_news.rename(columns={'fake_news': 'text'}, inplace=True)

In [ ]:
fake_news

,text,fake
0,Кремль принял решение не продлевать режим само...,1
1,Премьер-министр Израиля Биньямин Нетаньяху зая...,1
2,В период пандемии потребление дата-услуг в Рос...,1
3,Кремль принял решение не продлевать санкции пр...,1
4,В Москве умер 41 пациент с коронавирусом. Об э...,1
...,...,...
995,В Гатчине Ленинградской области в связи с панд...,1
996,В Москве умер 41 пациент с коронавирусом. Об э...,1
997,Участники Всемирного саммита по безопасности и...,1
998,Олимпийская чемпионка по фигурному катанию Еле...,1


## Склеиваем все новости в один датафрейм

In [ ]:
all_news = pd.concat([news[['text', 'fake']], fake_news], axis=0)

In [ ]:
all_news = all_news[['text', 'fake']]

In [ ]:
all_news

,text,fake
0,В 1930-е годы Советский Союз охватила лихорадк...,0
1,Олимпийская чемпионка по фигурному катанию Али...,0
2,Российский врач-диетолог Римма Мойсенко объясн...,0
3,В 2019 году телеканал «Ю» запустил адаптацию з...,0
4,Актер Михаил Ефремов систематически употреблял...,0
...,...,...
995,В Гатчине Ленинградской области в связи с панд...,1
996,В Москве умер 41 пациент с коронавирусом. Об э...,1
997,Участники Всемирного саммита по безопасности и...,1
998,Олимпийская чемпионка по фигурному катанию Еле...,1


## Загружаем модель

Список предобученных моделей: [https://huggingface.co/transformers/pretrained_models.html](https://huggingface.co/transformers/pretrained_models.html)

In [ ]:
# DistilBERT:

## задаем саму модель
model_class = transformers.DistilBertModel

## токенайзер к ней (для некоторых моделей токенайзер будет отличаться, см.
## в документации к каждой модели конкретно)
tokenizer_class = transformers.DistilBertTokenizer

## загружаем веса для моделей
pretrained_weights = 'distilbert-base-uncased'

## создаем объекты токенизатора для и модели
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
## зададим максимальную длину последовательности
max_len = 200

In [ ]:
# применяем токенизатор: 
# -≥ add_special_tokens = добавляем служебные токены
# -≥ truncation = обрезаем по максимальной длине
# -≥ max_length = максимальная длина последовательности
tokenized = all_news['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, 
                                                                      truncation=True, 
                                                                      max_length=max_len)))

In [ ]:
# посмотрим на результат токенизатора
tokenized.values[:2]

array([list([101, 1182, 4479, 1011, 1185, 1183, 14150, 29742, 29113, 1196, 19259, 15290, 22919, 29747, 23925, 15414, 1196, 14150, 29757, 29744, 1193, 29750, 25529, 10260, 22919, 10325, 29436, 10260, 1190, 10325, 29750, 14150, 16856, 10260, 29742, 28598, 1517, 1182, 1184, 15290, 29747, 17432, 22919, 10325, 29436, 15290, 22919, 10325, 15290, 1181, 29748, 16856, 18947, 14150, 10325, 1188, 18947, 29742, 29748, 29747, 22919, 16856, 10325, 10260, 29436, 10325, 29744, 10260, 29751, 15414, 1194, 19259, 29747, 29757, 29742, 29748, 1183, 16856, 15290, 29745, 15290, 29436, 10325, 1196, 14150, 14150, 29740, 29754, 15290, 18947, 23483, 1193, 1192, 19259, 29113, 29750, 1197, 16856, 29748, 29742, 19259, 29113, 29750, 1194, 14150, 29742, 25529, 10325, 29741, 10260, 29750, 1194, 16856, 14150, 29747, 22919, 14150, 29741, 14150, 1192, 10260, 16856, 14150, 29742, 10260, 1195, 10260, 29742, 10325, 1194, 14150, 29747, 22919, 16856, 14150, 15290, 18947, 23483, 1189, 14150, 29745, 29745, 29748, 18947, 10325, 

In [ ]:
# проверим длины последовательностей: максимальная будет 200, 
# но какие-то будут меньше
for i in tokenized.values[10:20]:
    print(len(i), end=' ')


200 200 200 200 200 200 200 200 200 200 

In [ ]:
# чтобы дополнить последовательности до фиксированной длины, допишем нули к 
# коротким последовательностям 
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [ ]:
# проверим
# теперь все последовательности должны быть одинаковой длины - max_len
for i in padded[:10]:
    print(len(i), end=' ')

200 200 200 200 200 200 200 200 200 200 

#### Masking
Обозначим, где мы добавили маску, чтобы модель не учитывала эту область при обработке

In [ ]:
padded[:1]

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask[1]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])

In [ ]:
# чтобы attention не считался для позиций, где установлены фиктивные
# нулевые токены, мы делаем для него маску

print(padded[1])
print(attention_mask[1])

In [ ]:
# переводим в тензоры
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

In [ ]:
input_ids.shape, attention_mask.shape

(torch.Size([3048, 200]), torch.Size([3048, 200]))

In [ ]:
import tqdm

In [ ]:
model.to(device)

In [ ]:
# f_feat = open("features.txt", "a")


In [ ]:

batch_size = 100
batch_num = len(all_news) // batch_size


for i in tqdm.notebook.tqdm(range(batch_num +1)):
  batch_ids = input_ids[i*batch_size:(i + 1)*batch_size, :]
  batch_attn_msk = attention_mask[i*batch_size:(i + 1)*batch_size, :]
  batch_ids, batch_attn_msk = batch_ids.to(device), batch_attn_msk.to(device)
  
# Получаем выход модели (нам оттуда нужно не все)
  with torch.inference_mode():
      last_hidden_states = model(batch_ids, attention_mask=batch_attn_msk)
  # features = last_hidden_states[0][:,0,:].cpu().numpy()
  # np.savetxt('batch_feat.txt', features)

  features = last_hidden_states[0][:,0,:].to(device)#.numpy()
  np.savetxt((f'test/test_{i}.txt'), features.cpu().numpy())
  
  # f = open('batch_feat.txt', 'r')
  # f_feat.write(f.read())

  0%|          | 0/56 [00:00<?, ?it/s]

In [ ]:
import os
dirname = '/content/test/'
files = os.listdir(dirname)
i = 1
full_df = pd.DataFrame()

for file in files:
  read_txt = pd.read_csv(f'/content/test/{file}', sep = ' ', header=None)
  full_df = pd.concat([full_df, read_txt], axis=0)
  i += 1

full_df

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.222723,0.166132,-0.100101,-0.324785,-0.027221,-0.041195,0.246974,0.137662,-0.193503,-0.129046,...,0.331069,-0.573177,0.299321,-0.207125,0.418603,-0.670402,-0.022335,-0.194791,0.819521,0.667621
1,-0.259144,0.125450,-0.084132,-0.321300,0.009030,-0.056226,0.339022,0.097649,-0.180784,-0.037718,...,0.307328,-0.532404,0.319854,-0.292285,0.412927,-0.651900,-0.055689,-0.283102,0.770349,0.627820
2,-0.215916,0.200925,-0.064011,-0.295483,-0.073845,0.009221,0.198645,0.036582,-0.225337,-0.076471,...,0.309593,-0.482827,0.241062,-0.197507,0.401634,-0.651321,0.007797,-0.193371,0.759559,0.695034
3,-0.232374,0.101365,-0.041778,-0.281117,0.001608,-0.029088,0.164953,0.152773,-0.182717,-0.145667,...,0.321685,-0.502628,0.249350,-0.235291,0.425426,-0.662214,-0.012173,-0.169773,0.811495,0.648542
4,-0.218087,0.162913,-0.189839,-0.309662,0.019484,-0.010935,0.228338,0.080355,-0.246617,-0.062663,...,0.296344,-0.529430,0.331410,-0.189492,0.452200,-0.681343,-0.031436,-0.271209,0.754457,0.651724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-0.261508,0.165040,-0.162997,-0.381953,-0.035193,-0.070349,0.246705,0.128871,-0.174185,-0.137896,...,0.382171,-0.560627,0.324163,-0.241709,0.459255,-0.670060,0.004776,-0.281158,0.830089,0.656648
96,-0.274564,0.156305,-0.170683,-0.312916,0.016781,-0.001971,0.270548,0.182786,-0.224772,-0.077808,...,0.296066,-0.543525,0.292842,-0.183397,0.420119,-0.689050,-0.034486,-0.245273,0.779730,0.681661
97,-0.328449,0.117576,-0.160159,-0.293585,-0.017341,0.024553,0.216206,0.150396,-0.223976,-0.126412,...,0.362253,-0.515894,0.352691,-0.192952,0.423511,-0.601797,-0.084425,-0.311429,0.747816,0.667006
98,-0.287989,0.160563,-0.098237,-0.299874,-0.019199,-0.002120,0.208788,0.129323,-0.252529,-0.071762,...,0.287684,-0.495914,0.274637,-0.214055,0.397872,-0.644413,-0.067477,-0.282151,0.826843,0.642113


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
 !cp /content/features.txt /content/drive/MyDrive/features.txt

In [ ]:
# !cp /content/drive/MyDrive/features.txt /content/features.txt

#### Используем выход BERT для обучения классического классификатора

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(full_df, all_news['fake'], shuffle=True)

In [ ]:
train_features.shape, test_features.shape, train_labels.shape, test_labels.shape

((2286, 768), (762, 768), (2286,), (762,))

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
mnb = MultinomialNB()
lrc = LogisticRegression()

In [ ]:
clfs = {    
    'LogisticRegression' : lrc,
    'MultinomialNB' : mnb
}

In [ ]:
def get_scores(clfs, n_cv=2):
    for clf in clfs: 
        scores = cross_val_score(clfs[clf], full_df, all_news['fake'], n_jobs=4, cv=n_cv, scoring='accuracy')
                
        mean_score, std_score = np.mean(scores), np.std(scores)
        
        print(f''' 
        {clf}
        score: {mean_score:.3f}, std: {std_score:.5f}
       
        ''')

In [ ]:
get_scores(clfs, 2)

 
        LogisticRegression
        score: 0.819, std: 0.00000
       
        
 
        MultinomialNB
        score: nan, std: nan
       
        


In [ ]:
train_labels

In [ ]:
lrc.fit(train_features, train_labels)

LogisticRegression()

In [ ]:
import pickle
# save the model to disk
filename = 'linear_regression.sav'
pickle.dump(lrc, open(filename, 'wb'))

In [ ]:
!cp '/content/linear_regression.sav'  '/content/drive/MyDrive/linear_regression.sav'

# Fake or not?

In [ ]:
def check_fake(text):
  tokenized = tokenizer.encode(text, add_special_tokens=True, truncation=True, max_length=max_len)
  padded = np.array([tokenized + [0]*(max_len-len(tokenized)) ])
  attention_mask = np.where(padded != 0, 1, 0)
  input_id = torch.tensor(padded)  
  attention_mask = torch.tensor(attention_mask)
  input_id, attention_mask = input_id.to(device), attention_mask.to(device)
  
  with torch.inference_mode():
      last_hidden_states = model(input_id, attention_mask=attention_mask)

  features = last_hidden_states[0][:,0,:].to(device)#.numpy()
  np.savetxt((f'test.txt'), features.cpu().numpy())
  
  vecorized = pd.read_csv(f'test.txt', sep = ' ', header=None)
  
  if lrc.predict(vecorized)[0]:
    return 'fake'
  else:
    return 'real'
  # return lrc.predict(vecorized)



In [ ]:
test_news = '''Вступление в ЕС Сербии и самопровозглашенной республики Косово может стать нереализуемым без взаимного признания Белграда и Приштины. Условие для присоединения сербского государства к Евросоюзу назвал канцлер ФРГ Олаф Шольц во время рабочей поездки на Балканы, сообщает РИА Новости.'''

In [ ]:
check_fake(test_news)

'real'

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

In [ ]:
demo = gr.Interface(fn=check_fake, inputs='text',  outputs='text')

demo.launch(debug=True)